<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Признаки-умножают-на-обратимую-матрицу.-Изменится-ли-качество-линейной-регрессии?" data-toc-modified-id="Признаки-умножают-на-обратимую-матрицу.-Изменится-ли-качество-линейной-регрессии?-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии?</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span><ul class="toc-item"><li><span><a href="#Вывод" data-toc-modified-id="Вывод-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li></ul></div>

# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

In [1]:
import pandas as pd
import numpy as np
import random
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

state = np.random.RandomState(12345) # закрепляем весь рандом в проекте

## Загрузка данных

In [11]:
try:
    data = pd.read_csv('insurance.csv')
except:
    data = pd.read_csv('/datasets/insurance.csv')                       

In [14]:
def data_info(data):
    print('ОБЩАЯ ИНФОРМАЦИЯ', sep='\n')
    print(data.info()) 
    print('ПРОПУСКИ В ДАННЫХ', '', data.isna().sum(), '', sep='\n')
    print('ПОЛНЫЕ ДУБЛИКАТЫ В ДАННЫХ', '', data.duplicated().sum(), '', sep='\n')
    print('ПЕРВЫЕ 5 ЗАПИСЕЙ', sep='\n')
    display(data.head(5))
    print('Общая статистика:')
    display(data.describe())
    
data_info(data)

ОБЩАЯ ИНФОРМАЦИЯ
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB
None
ПРОПУСКИ В ДАННЫХ

Пол                  0
Возраст              0
Зарплата             0
Члены семьи          0
Страховые выплаты    0
dtype: int64

ПОЛНЫЕ ДУБЛИКАТЫ В ДАННЫХ

153

ПЕРВЫЕ 5 ЗАПИСЕЙ


,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


Общая статистика:


,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


## Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии?

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

In [15]:
# Выделим признаки и целевой признак
X = data.drop('Страховые выплаты', axis=1) # матрица признаков
y = data['Страховые выплаты'] # вектор целевого признака

*Первым делом посчитаем линейную регрессию в её изначальном виде чтобы было с чем сравнивать, а после будем проводить преобразования*

In [16]:
class LinearRegression:
    def fit(self, X_train, y_train):
        '''
        Input --> martrix: признаки - X и целевой признак - y
        Process: w - отвечает за формулу обучения и исполняет вышеописанную формулу
        self.w, self.w0 - параметры линейной регрессии
        '''
        
        X = np.concatenate((np.ones((X_train.shape[0], 1)), X_train), axis=1)
        y = y_train
        w = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y) 
        self.w = w[1:]
        self.w0 = w[0]
        
    def predict(self, X_test):
        '''
        Input --> martix: X_test - матрица признаков
        Process: функция использует формулу обучения линейной регрессии $a=Xw+w0$
        Output --> array: вектор предсказанных значений
        
        '''
        return X_test.dot(self.w) + self.w0

In [17]:
model = LinearRegression()
model.fit(X, y)
predictions = model.predict(X)
print(f'r2_score равен:{r2_score(y, predictions)}')

r2_score равен:0.4249455028666801


***Теперь исследуем процесс умножения матрицы признаков на обратимую матрицу***

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

**Ответ:** Не изменится.

**Обоснование:** 

Умножение матрицы на обратимую матрицу:

$$
Xm = XP 
$$

Предсказания:
$$
am = Xm * w
$$

Формула обучения:

$$
wm = (Xm^T Xm)^{-1} Xm^T y
$$

преобразования:

$$
wm = (Xm^T Xm)^{-1} Xm^T y =  
$$
$$
((XP)^T XP)^{-1} (XP)^T y =  
$$
$$
(X^TXP)^{-1} (P^T)^{-1} X^TP^T y => (P^T)^{-1} P^T = E => 
$$
$$
P^{-1}(X^TX) X^T y = 
$$
$$
P^{-1} w
$$

Итог:
$$
wm = P^{-1} w
$$
Условие:
$$
am = Xm wm
$$
Подстановка:
$$
am = XP P^{-1} w = 
$$
$$
XE w = X w
$$


Итог подстановки:
$$
am = a
$$

## Алгоритм преобразования

**Алгоритм преобразования:**
1. Генерация случайной матрицы 4x4
2. Вычисление обратной матрицы 4x4
3. Умножение признаков на данную матрицу 

**Алгоритм проверки корректности полученных матриц:**

4. Восстановление признаков по данной матрице 
5. Проверка одинаковости оригинальной и восстановленной матриц

In [20]:
# Создадим массив данных размером M x N
primal_matrix = np.random.normal(1,100, size=(5000, 4))
print(primal_matrix.shape)
primal_matrix

(5000, 4)


array([[ -21.69986051,   -8.41892989, -128.2745335 ,   -7.51756325],
       [ -14.95827583,  127.64280256,   19.04730585,   -3.53227538],
       [-153.09048668,   84.70149036, -114.39598349,  -19.57143544],
       ...,
       [  27.59924007,   97.03109854,   28.12783471, -198.70804586],
       [ -66.18057597,  -47.43198007,  120.99963914, -144.97953007],
       [  34.98938662,  139.98243349,   70.57019912,   77.26398153]])

In [24]:
def rnd_matrix(n):
    random_matrix = np.random.normal(size=(n, n))
    try:
        np.linalg.inv(random_matrix) # выполняется операция создания обратной матрицы
    except LinAlgError: # если возникает ошибка, т.е. обратной матрицы не существует, выполняется еще одна попытка
        rnd_matrix(n)
    return random_matrix

In [26]:
# Генерация случайной матрицы N x N
generate_matrix = rnd_matrix(primal_matrix.shape[1])
generate_matrix

array([[-0.64076666, -0.74886588,  0.38583256,  0.09987712],
       [ 1.10084007,  0.83961274,  2.2527335 ,  1.34601667],
       [-1.36909495, -0.01321899, -0.05771433, -0.9321204 ],
       [ 0.1133841 , -0.20128134,  1.58677193, -0.8801263 ]])

In [27]:
# Вычисление обратной матрицы 4x4
inverse_matrix = generate_matrix @ np.linalg.inv(generate_matrix)
inverse_matrix

array([[ 1.00000000e+00, -2.77555756e-17,  1.11022302e-16,
        -8.32667268e-17],
       [-1.04083409e-16,  1.00000000e+00, -1.11022302e-16,
         0.00000000e+00],
       [ 0.00000000e+00,  5.55111512e-17,  1.00000000e+00,
         0.00000000e+00],
       [ 5.55111512e-17, -5.55111512e-17,  5.55111512e-17,
         1.00000000e+00]])

In [28]:
# умножаем оригинальную матрицу на ключ
new_matrix = primal_matrix @ generate_matrix
new_matrix

array([[ 179.40429579,   12.39044897,  -31.86349724,  112.68437437],
       [ 123.62100272,  118.83146071,  275.06960964,  155.66981617],
       [ 345.73794925,  191.2122565 ,  107.28947377,  222.57554438],
       ...,
       [  28.09104035,  100.42481668,  -87.69383638,  282.03165666],
       [-191.90708539,   37.31806138, -369.41909385,  -55.6400959 ],
       [  43.82144339,   74.8440128 ,  447.37056711,   58.13134394]])

**Теперь данные сильно отличаются от тех что были ранее**

In [29]:
# восстановление первоначальной матрицы
returned_matrix = new_matrix @ np.linalg.inv(generate_matrix)
returned_matrix

array([[ -21.69986051,   -8.41892989, -128.2745335 ,   -7.51756325],
       [ -14.95827583,  127.64280256,   19.04730585,   -3.53227538],
       [-153.09048668,   84.70149036, -114.39598349,  -19.57143544],
       ...,
       [  27.59924007,   97.03109854,   28.12783471, -198.70804586],
       [ -66.18057597,  -47.43198007,  120.99963914, -144.97953007],
       [  34.98938662,  139.98243349,   70.57019912,   77.26398153]])

In [30]:
# проверка на одинаковость получившихся матриц
np.allclose(returned_matrix, primal_matrix)

True

**Вывод**

Алгоритм преобразования работает без нарекательств в соответствии с расчётами

## Проверка алгоритма

In [31]:
# разбиваем датасет на выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4)

In [32]:
# проверка размерности
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(3000, 4)
(2000, 4)
(3000,)
(2000,)


In [33]:
# экземпляр моделей линейной регрессии для зашифрованных и не зашифрованных выборок
model_1, model_2 = LinearRegression(), LinearRegression()

In [34]:
# процесс шифрования
crypto_X_train = X_train @ generate_matrix
crypto_X_test = X_test @ generate_matrix
crypto_X_train

,0,1,2,3
49,-50077.938279,-460.507056,-2047.681322,-34078.798131
2516,-51705.327074,-464.816543,-2083.813482,-35179.378498
1231,-45296.660817,-423.441853,-1861.861399,-30833.357419
1544,-39676.119176,-362.561591,-1615.810582,-26998.721176
3633,-71571.281471,-666.913514,-2950.491841,-48709.416301
...,...,...,...,...
4449,-38845.508796,-347.621269,-1562.108319,-26426.354785
2640,-57596.605416,-524.523330,-2341.531030,-39189.674118
1461,-49383.136553,-446.049019,-1997.497759,-33598.297763
2385,-44730.302018,-402.984970,-1804.187718,-30432.660578


Данные разительно отличаются от изначальных

In [35]:
# обучение и предсказания моделей
model_1.fit(X_train, y_train)
model_2.fit(crypto_X_train, y_train)
predicted_original = model_1.predict(X_test)
predicted_crypto = model_2.predict(crypto_X_test)

In [36]:
# результаты
pd.DataFrame({'R2 for original': [r2_score(y_test, predicted_original)], 
              'R2 for crypto': [r2_score(y_test, predicted_crypto)]}, index=['Result'])

,R2 for original,R2 for crypto
Result,0.424358,0.424358


### Вывод 

* В ходе работы данные были проверены - датасет в предобработке не нуждался.

* Были сделаны расчёты по обоснованию алгоритма шифрования данных и подтверждены соответствующими расчётами.

* Датасет был разбит на тренировочный и тестовый.

* Обучены модели на зашифрованных и незашифрованных данных.

* Результаты R2_score практически не отличались друг от друга